# 🧩 ETL – Anonimização de Dados Regulatórios 

## 🎯 Objetivo do script

- 📥 Ler a base bruta de processos regulatórios (MEC/SERES)
- 🕵️‍♂️ Anonimizar dados sensíveis de identificação  
  - Mantenedora (código, nome, CNPJ)  
  - IES (código, nome)
- 📤 Gerar uma versão tratada e segura para uso em portfólio (GitHub)


In [1]:
import pandas as pd
import hashlib
from pathlib import Path

In [2]:
# ----------------------------------------
# Arquivos entrada e saida
# ----------------------------------------
ARQUIVO_ORIGINAL = "TOTAL_01_12.xlsx"

ARQUIVO_ATE_2018 = "TOTAL_01_12_ate_2018_anonimizado.xlsx"
ARQUIVO_DESDE_2019 = "TOTAL_01_12_desde_2019_anonimizado.xlsx"

In [3]:
# ----------------------------------------
# Auxiliar
# ----------------------------------------
def gerar_cnpj_fake(seed: str) -> str:
    h = hashlib.sha256(seed.encode("utf-8")).hexdigest()
    digits = "".join(c for c in h if c.isdigit())
    if len(digits) < 14:
        digits = (digits * 14)[:14]
    else:
        digits = digits[:14]
    return f"{digits[0:2]}.{digits[2:5]}.{digits[5:8]}/{digits[8:12]}-{digits[12:14]}"


In [4]:
# ----------------------------------------
# 1. Leitura
# ----------------------------------------
print(f"Lendo arquivo: {ARQUIVO_ORIGINAL}")
df = pd.read_excel(ARQUIVO_ORIGINAL)

Lendo arquivo: TOTAL_01_12.xlsx


In [5]:
# Garantir numericidade da coluna de split
df["Ano do Protocolo"] = pd.to_numeric(df["Ano do Protocolo"], errors="coerce")

# ----------------------------------------
# 2. SPLIT por ano antes da anonimização
# ----------------------------------------
df_ate_2018 = df[df["Ano do Protocolo"] < 2019].copy()
df_desde_2019 = df[df["Ano do Protocolo"] >= 2019].copy()

print("🔀 Linhas até 2018:", len(df_ate_2018))
print("🔀 Linhas desde 2019:", len(df_desde_2019))

🔀 Linhas até 2018: 360230
🔀 Linhas desde 2019: 299182


In [6]:
# ----------------------------------------
# Função para anonimizar um dataframe
# ----------------------------------------
def anonimizar(df_local):
    
    # ---- Anonimização MANTENEDORA ----
    m_key = df_local["Código Mantenedora"].astype(str)
    m_unique = m_key.dropna().unique()
    m_map = {orig: i + 1 for i, orig in enumerate(sorted(m_unique))}

    df_local["MANT_ID_FAKE"] = m_key.map(
        lambda x: f"MANT_{m_map[x]:04d}" if x in m_map else pd.NA
    )
    df_local["MANT_NOME_FAKE"] = df_local["MANT_ID_FAKE"].map(
        lambda x: f"Mantenedora {str(x).split('_')[-1]}" if pd.notna(x) else pd.NA
    )
    df_local["CNPJ_MANTENEDORA_FAKE"] = df_local["MANT_ID_FAKE"].map(
        lambda x: gerar_cnpj_fake(str(x)) if pd.notna(x) else pd.NA
    )

    # ---- Anonimização IES ----
    ies_key = df_local["Código da IES"].astype(str)
    ies_unique = ies_key.dropna().unique()
    ies_map = {orig: i + 1 for i, orig in enumerate(sorted(ies_unique))}

    df_local["IES_ID_FAKE"] = ies_key.map(
        lambda x: f"IES_{ies_map[x]:05d}" if x in ies_map else pd.NA
    )
    df_local["IES_NOME_FAKE"] = df_local["IES_ID_FAKE"].map(
        lambda x: f"IES {str(x).split('_')[-1]}" if pd.notna(x) else pd.NA
    )

    # ---- Remover dados sensíveis ----
    colunas_sensiveis = [
        "Código Mantenedora",
        "Nome Mantenedora",
        "CNPJ Mantenedora",
        "Código da IES",
        "Nome da IES",
        "Nome Técnico",
        "Sinalizações vigente do Processo: Usuário Inclusão",
    ]

    colunas_presentes = [c for c in colunas_sensiveis if c in df_local.columns]
    df_local = df_local.drop(columns=colunas_presentes)

    # ---- Datas e Derivações ----
    for col in ["Data", "Data do Último Ato"]:
        if col in df_local.columns:
            df_local[col] = pd.to_datetime(df_local[col], errors="coerce")

    if {"Data", "Data do Último Ato"}.issubset(df_local.columns):
        df_local["tempo_tramitacao_dias"] = (
            df_local["Data do Último Ato"] - df_local["Data"]
        ).dt.days

    if {"Vagas Solicitadas Processo", "Vagas Autorizadas Cadastro"}.issubset(df_local.columns):
        df_local["dif_vagas_processo_cadastro"] = (
            df_local["Vagas Solicitadas Processo"].fillna(0)
            - df_local["Vagas Autorizadas Cadastro"].fillna(0)
        )
        df_local["tem_divergencia_vagas"] = df_local[
            "dif_vagas_processo_cadastro"
        ].ne(0).astype(int)

    return df_local


In [7]:
# ----------------------------------------
# 3. Aplicar anonimização separada para cada arquivo
# ----------------------------------------
print("🧩 Anonimizando parte até 2018...")
df_ate_2018_anon = anonimizar(df_ate_2018)

print("🧩 Anonimizando parte desde 2019...")
df_desde_2019_anon = anonimizar(df_desde_2019)

🧩 Anonimizando parte até 2018...
🧩 Anonimizando parte desde 2019...


In [8]:

# ----------------------------------------
# 4. Salvar os arquivos finais
# ----------------------------------------
df_ate_2018_anon.to_excel(ARQUIVO_ATE_2018, index=False)
df_desde_2019_anon.to_excel(ARQUIVO_DESDE_2019, index=False)

print("✅ Arquivos anonimizados e divididos salvos com sucesso!")
print(f"📁 {ARQUIVO_ATE_2018}")
print(f"📁 {ARQUIVO_DESDE_2019}")


✅ Arquivos anonimizados e divididos salvos com sucesso!
📁 TOTAL_01_12_ate_2018_anonimizado.xlsx
📁 TOTAL_01_12_desde_2019_anonimizado.xlsx
